In [1]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}


print('DONE')

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 131331 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.6-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.6-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.6-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=ht

**Make dir:**

In [2]:
!mkdir -p drive
!google-drive-ocamlfuse -o nonempty drive

print('DONE')

DONE


**Change Working Path:**

In [3]:
import os

path = "/content/drive/HourGlass_torch"
# path = "/content/gdrive/My Drive/HourGlass_torch"
os.chdir(path)
os.listdir(path)

print('DONE')

DONE


Begin Net
===
**Import Packages:**

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as datatorch

import numpy as np
import matplotlib.pyplot as plt
import json
import os
import math

from utils import imgutils
from models.hourglass import hg as hg_torch
from models.hourglass2 import hourglassnet as hgnet_torch
from losses.jointsmseloss import JointsMSELoss
from datasets.dataset_torch_new2 import DatasetTorch

import os

print('DONE')

DONE


**Conctruct Dataset:**

In [0]:
# ================================== Construct dataset ==================================
num_bcsz = 16

# ds_torch = DatasetTorch(use_flip=False, use_rand_color=True)  # Use True of rand_flip for this will cause error when inner default is False
ds_torch = DatasetTorch(use_flip=False, use_rand_color=True, use_randflipLR_inner=True)
data_loader = datatorch.DataLoader(ds_torch, batch_size=num_bcsz, shuffle=True)

print('Construct dataset DONE')

**Construct Model:**

In [0]:
# ================================== Construct model ==================================
device = torch.device('cuda')
# device = torch.device('cpu')
learning_rate = 1e-3

# net_hg_torch = hg_torch(num_stacks=4, num_blocks=1, num_classes=16).to(device)
net_hg_torch = hgnet_torch(num_stacks=2, num_blocks=1, num_classes=16, num_features=64).to(device)
optimizer = torch.optim.RMSprop(net_hg_torch.parameters(), lr=learning_rate)
# optimizer = torch.optim.Adam(net_hg_torch.parameters(), lr=learning_rate, weight_decay=0)
criteon = JointsMSELoss(use_target_weight=True).to(device)

print('Construct model DONE')

***Load Checkpoint and Recap Model, Optimizer, etc:***

In [5]:
# ================================== Construct dataset ==================================
num_bcsz = 16

# ds_torch = DatasetTorch(use_flip=False, use_rand_color=True)  # Use True of rand_flip for this will cause error when inner default is False
ds_torch = DatasetTorch(use_flip=False, use_rand_color=True, use_randflipLR_inner=True)
data_loader = datatorch.DataLoader(ds_torch, batch_size=num_bcsz, shuffle=True)

print('Construct dataset DONE')

num_epoch = 20
num_setsize = ds_torch.__len__()

# ================================== Reconstruct Net, Optim, Criteon ==================================
device = torch.device('cuda')
# device = torch.device('cpu')
learning_rate = 1e-3

# net_hg_torch = hg_torch(num_stacks=8, num_blocks=1, num_classes=16)
# print('Reconstruct Model DONE')
# optimizer = torch.optim.RMSprop(net_hg_torch.parameters(), lr=learning_rate)
# print('Reconstruct Optimizer DONE')
criteon = JointsMSELoss(use_target_weight=True).to(device)
print('Reconstruct Criteon DONE')


# ================================== Reload Net, Optim ==================================
suffix = 'allEPOCH14'  # saved suffix to load
path_ckpt_torch = 'models/ckpt_hg_torch_' + str(suffix) + '.tar'
checkpoint = torch.load(path_ckpt_torch)

# net_hg_torch = hg_torch(num_stacks=4, num_blocks=1, num_classes=16)
net_hg_torch = hgnet_torch(num_stacks=2, num_blocks=1, num_classes=16, num_features=64).to(device)
net_hg_torch = net_hg_torch.to(device)
net_hg_torch.load_state_dict(checkpoint['model_state_dict'])
print('Reconstruct Model DONE')
optimizer = torch.optim.RMSprop(net_hg_torch.parameters(), lr=learning_rate)
# optimizer = torch.optim.Adam(net_hg_torch.parameters(), lr=learning_rate, weight_decay=0)
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
print('Reconstruct Optimizer DONE')
LOSS = checkpoint['loss']
EPOCH = checkpoint['epoch']
STEP = checkpoint['step']

net_hg_torch.train()

print('===============CHECKPOINT LOADED===============')
print('CKPT LOSS', LOSS)
print('CKPT EPOCH', EPOCH)
print('CKPT STEP', STEP)

# EPOCH = 0  # Reload EPOCH as file name
print('UPDATE EPOCH', EPOCH)

# Calculate EPOCH and STEP left to go
# EPOCH_left = 20-(EPOCH)  # If not all epoch
EPOCH_left = 20-(EPOCH+1)  # If all epoch
STEP_left = math.ceil(num_setsize / num_bcsz)-(STEP+1)
print('LEFT CKPT EPOCH', EPOCH_left)
print('LEFT CKPT STEP', STEP_left)

Construct dataset DONE
Reconstruct Criteon DONE
Reconstruct Model DONE
Reconstruct Optimizer DONE
===============CHECKPOINT LOADED===============
CKPT LOSS tensor(0.0008, device='cuda:0', requires_grad=True)
CKPT EPOCH 14
CKPT STEP 1390
UPDATE EPOCH 14
LEFT CKPT EPOCH 5
LEFT CKPT STEP 0


**Train and Save Checkpoint:**

In [0]:
# ================================== Train ==================================
# # First time train
# num_epoch = 20
# num_setsize = ds_torch.__len__()
# EPOCH = -1


# Reload CKPT and train
num_epoch = EPOCH_left



print('num_setsize', num_setsize)
target_weight = np.array([[1.2, 1.1, 1, 1, 1.1, 1.2, 1, 1, 1, 1, 1.2, 1.1, 1, 1, 1.1, 1.2]])
target_weight = torch.from_numpy(target_weight).to(device).float()

print('data_loader', data_loader)

plt.ion()
for i in range(num_epoch):
    for step, (img, heatmaps, pts) in enumerate(data_loader):
        # To GPU
        img, heatmaps = img.to(device), heatmaps.cuda()

        # All dtype change to float
        img, heatmaps = img.float(), heatmaps.float()

        print('')
        print('EPOCH', str(num_epoch), '/', i + 1, ' ||  STEP', math.ceil(num_setsize / num_bcsz), '/', step + 1)
        
        heatmaps_pred_list = net_hg_torch(img)
        
#         # Intermediate Supervision
#         loss = torch.zeros([]).to(device)
#         for ele_heatmaps_pred in heatmaps_pred_list:
#             ele_loss = criteon(ele_heatmaps_pred, heatmaps, target_weight)
#             loss = loss + ele_loss
#         loss = loss / len(heatmaps_pred_list)
#         heatmaps_pred_final = heatmaps_pred_list[-1]
        
        # Only final loss, NO intermediate supervision
        heatmaps_pred_final = heatmaps_pred_list[-1]
        loss = criteon(heatmaps_pred_final, heatmaps, target_weight)
        
        
        print('LOSS:', loss)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        
        # Show last layer output
        heatmaps_pred_copy = heatmaps_pred_final[1]
        heatmaps_copy = heatmaps[1]
        img_copy = img[1]

        # Show pred heatmaps
        heatmaps_pred_np = heatmaps_pred_copy.detach().cpu().numpy()
        heatmaps_pred_np = np.transpose(heatmaps_pred_np, (1, 2, 0))
        heatmaps_np = heatmaps_copy.detach().cpu().numpy()
        heatmaps_np = np.transpose(heatmaps_np, (1, 2, 0))
        img_np = img_copy.detach().cpu().numpy()
        img_np = np.transpose(img_np, (1, 2, 0))
        
        # In case too many figs to print
        if step % 5 == 0:
            imgutils.show_heatmaps(img_np, heatmaps_pred_np, num_fig=1)
            imgutils.show_heatmaps(img_np, heatmaps_np, num_fig=2)
#             plt.pause(0.1)
        print('===================================================')
        
        if (step % 300 == 0) and (step > 0):
            suffix_epoch = i+EPOCH+1
            suffix_step = step
            path_ckpt_torch = 'models/ckpt_hg_torch_EPOCH' + str(suffix_epoch) + 'STEP' + str(
                suffix_step) + '.tar'
            torch.save({
            'epoch': i+EPOCH+1,
            'step': step,
            'model_state_dict': net_hg_torch.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss
            }, path_ckpt_torch)
            print('===============CHECKPOINT PARAMS SAVED===============')
        del img, heatmaps, pts, heatmaps_pred_copy, heatmaps_copy,  img_copy, heatmaps_pred_np, heatmaps_np, img_np

    # ================================== Save checkpoint each epoch ==================================
    suffix_epoch = i+EPOCH+1
    suffix_step = 0
    path_ckpt_torch = 'models/ckpt_hg_torch_allEPOCH' + str(suffix_epoch) + '.tar'
    torch.save({
            'epoch': i+EPOCH+1,
            'step': step,
            'model_state_dict': net_hg_torch.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss
            }, path_ckpt_torch)
    print('===============CHECKPOINT PARAMS SAVED===============')
plt.ioff()

print('Training DONE')

Output hidden; open in https://colab.research.google.com to view.